In [1]:
import requests
import json
import pandas as pd
import edgar 
import numpy as np
from bs4 import BeautifulSoup
import datetime as dt
import pandas_datareader.data as web

In [2]:
sym= pd.read_csv('edgar/symbol_map.csv')
comp=sym['COMPANY NAME']

In [3]:
df1= pd.read_csv('edgar/2008-QTR1.tsv', sep='|',  lineterminator='\n', header=None,names=['cki', 'name', 'form', 'date', 'l1','l2','tick']) 
df2= pd.read_csv('edgar/2008-QTR2.tsv', sep='|',  lineterminator='\n', header=None,names=['cki', 'name', 'form', 'date', 'l1','l2','tick']) 
df3= pd.read_csv('edgar/2008-QTR3.tsv', sep='|',  lineterminator='\n', header=None,names=['cki', 'name', 'form', 'date', 'l1','l2','tick']) 
df4= pd.read_csv('edgar/2008-QTR4.tsv', sep='|',  lineterminator='\n', header=None,names=['cki', 'name', 'form', 'date', 'l1','l2','tick']) 

df= df1[df1['form']=='10-K']
df= df.append(df2[df2['form']=='10-K'], ignore_index=True)
df= df.append(df3[df3['form']=='10-K'], ignore_index=True)
df= df.append(df4[df4['form']=='10-K'], ignore_index=True)

l1=[i for i in df['name'].values ]

for i in l1:
    if i in comp.values:
        df['tick'][l1.index(i)]=sym['SYMBOL'][list(comp.values).index(i)]
df.dropna(inplace=True)
df['form']=np.nan
df.rename(columns={"form": "F_Score"},inplace=True)
selectedreport='10-K'

E:\DOWNLOAD\anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
E:\DOWNLOAD\anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,cki,name,F_Score,date,l1,l2,tick
0,1000180,SANDISK CORP,NaN,2008-02-25,edgar/data/1000180/0000950134-08-003259.txt,edgar/data/1000180/0000950134-08-003259-index....,sndk
1,1000209,MEDALLION FINANCIAL CORP,NaN,2008-03-13,edgar/data/1000209/0001193125-08-055644.txt,edgar/data/1000209/0001193125-08-055644-index....,mfin
4,1000230,OPTICAL CABLE CORP,NaN,2008-01-29,edgar/data/1000230/0001193125-08-014289.txt,edgar/data/1000230/0001193125-08-014289-index....,occ
9,1000683,BLONDER TONGUE LABORATORIES INC,NaN,2008-03-31,edgar/data/1000683/0000950159-08-000585.txt,edgar/data/1000683/0000950159-08-000585-index....,bdr
10,1000694,NOVAVAX INC,NaN,2008-03-17,edgar/data/1000694/0000950133-08-001105.txt,edgar/data/1000694/0000950133-08-001105-index....,nvax
...,...,...,...,...,...,...,...
8734,934550,SEMITOOL INC,NaN,2008-12-12,edgar/data/934550/0000934550-08-000086.txt,edgar/data/934550/0000934550-08-000086-index.h...,smtl
8736,936395,CIENA CORP,NaN,2008-12-23,edgar/data/936395/0000950133-08-004117.txt,edgar/data/936395/0000950133-08-004117-index.h...,cien
8737,936528,WASHINGTON FEDERAL INC,NaN,2008-11-28,edgar/data/936528/0001193125-08-245206.txt,edgar/data/936528/0001193125-08-245206-index.h...,wafd
8743,947661,DIEDRICH COFFEE INC,NaN,2008-10-08,edgar/data/947661/0001193125-08-208074.txt,edgar/data/947661/0001193125-08-208074-index.h...,ddrx


In [25]:
for i in range(len(df)):
    url = 'https://www.sec.gov/Archives/' + df['l2'].values[i][:-1]
    df2 = pd.read_html(url)
    document_index = df2[0][df2[0]['Type'].notna()]

    document_name = document_index[document_index['Type'].str.contains(selectedreport)]['Document'].str.split(' ')[0][0]

    report_formatted = url.replace('-','').replace('index.html','')
    url =  report_formatted + '/' + document_name
    
    df2=pd.read_html(url)
    Balance_Sheet= Income_Sheet = Cash_flow= pd.DataFrame()
    flag1=flag2=flag3=0
    for item in df2:
        if item.dtypes.values[0]=='O':
            BS = (item[item.columns[0]].str.contains('Total assets') | 
                  item[item.columns[0]].str.contains('Total Assets'))
            BS2= (item[item.columns[0]].str.contains('Total stockholders') | 
                  item[item.columns[0]].str.contains('Total shareholders')) 
            BS3= (item[item.columns[0]].str.contains('Total current liabilities')|
                  item[item.columns[0]].str.contains('Current liabilities'))
            BS4= (item[item.columns[0]].str.contains('Retained earnings')| 
                  item[item.columns[0]].str.contains('Accumulated deficit'))
            BS5= (item[item.columns[0]].str.contains('Total current assets'))
            BS6= (item[item.columns[0]].str.contains('long-term debt')| 
                  item[item.columns[0]].str.contains('Long term debt')| 
                  item[item.columns[0]].str.contains('non-current'))
            
            IS = (item[item.columns[0]].str.contains('Net income')|
                  item[item.columns[0]].str.contains('Net loss'))
            IS2 = (item[item.columns[0]].str.contains('Total revenues') ) 
            
            CF = (item[item.columns[0]].str.contains('Operating activities') | 
                  item[item.columns[0]].str.contains('operating activities') | 
                  item[item.columns[0]].str.contains('by operating activities') | 
                  item[item.columns[0]].str.contains('in operating activities'))
            
            if flag1==0 and BS.any() and BS2.any() and BS3.any() and BS4.any() and BS5.any()  :
                Balance_Sheet = item
                flag1=1
                continue
                
            if flag2==0 and IS.any() and IS2.any() :
                Income_Sheet = item
                flag2=1
                continue
            
            if flag3==0 and CF.any() :
                Cash_flow = item
                flag3=1
                continue
    
    Pscore=0
    
    if len(Balance_Sheet)!=0 and len(Cash_flow)!=0  :
        
        Balance_Sheet = Balance_Sheet[Balance_Sheet[Balance_Sheet.columns[0]].notna()]
        Cash_flow = Cash_flow[Cash_flow[Cash_flow.columns[0]].notna()]
        
        assets = Balance_Sheet[(Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Total assets') |
                           Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Total Assets') ) ]
        assets = list(filter(('$').__ne__, list(assets.T.dropna().T.values[0])))
        
        current_assets = Balance_Sheet[(Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Total current assets') |
                           Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Total ssets') ) ]
        current_assets =list(filter(('$').__ne__, list(current_assets.T.dropna().T.values[0])))
        
        current_lia = Balance_Sheet[(Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Total current liabilities') |
                           Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Total ssets') ) ]
        current_lia =list(filter(('$').__ne__, list(current_lia.T.dropna().T.values[0])))
        
        equity= Balance_Sheet[(Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Total stockholders') |
                           Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Total shareholders') ) ]
        equity= list(filter(('$').__ne__, list(equity.T.dropna().T.values[0])))
        
        retained = Balance_Sheet[(Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Retained earnings') |
                           Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Accumulated deficit') ) ]
        retained = list(filter(('$').__ne__, list(retained.T.dropna().T.values[0])))
        
        oper_act = Cash_flow[(Cash_flow[Cash_flow.columns[0]].str.contains('by operating activities') |
                             Cash_flow[Cash_flow.columns[0]].str.contains('in operating activities')|
                             Cash_flow[Cash_flow.columns[0]].str.contains('operating activities')|
                             Cash_flow[Cash_flow.columns[0]].str.contains('Operating activities')) ]
        oper_act = list(filter(('$').__ne__, list(pd.DataFrame(oper_act.values[-1].T).dropna().T.values[0])))
        
        long_debt = Balance_Sheet[(Balance_Sheet[Balance_Sheet.columns[0]].str.contains('long-term debt') |
                           Balance_Sheet[Balance_Sheet.columns[0]].str.contains('Long term debt')  |
                           Balance_Sheet[Balance_Sheet.columns[0]].str.contains('non-current') ) ]
        if len(long_debt)!=0:
            long_debt =list(filter(('$').__ne__, list(long_debt.T.dropna().T.values[0])))
            long_debt=[i.replace('(','-').replace(')','').replace(',','') for i in long_debt]
            long_debt=[i for i in long_debt if i!='']
            long_debt=[i for i in long_debt[1:] if i[0] in ['1','2','3','4','5','6','7','8','9','0','-']]
            Pscore+=1 if float(long_debt[1])>float(long_debt[2]) else 0
        
        lists= [oper_act, assets, equity, retained, current_assets,current_lia]
        lists2=[]    
        for items in lists:
            items=[i.replace('(','-').replace(')','').replace(',','') for i in items]
            items=[items[i] for i in range(len(items)) if items[i]!='' and items[i] not in items[i+1:]]
            lists2.append(items)
        [oper_act, assets, equity, retained, current_assets,current_lia]=lists2
        
        Pscore+=1 if float(current_assets[1])/float(current_lia[1])>float(current_assets[2])/float(current_lia[2]) else 0

        Pscore+=1 if float(equity[2])-float(retained[2])> float(equity[1]) - float(retained[1]) else 0

        
    if len(Income_Sheet)!=0:
        Income_Sheet = Income_Sheet[Income_Sheet[Income_Sheet.columns[0]].notna()]
        
        net_inc = Income_Sheet[(Income_Sheet[Income_Sheet.columns[0]].str.contains('Net income')|
                                Income_Sheet[Income_Sheet.columns[0]].str.contains('Net loss') ) ]
        net_inc = list(filter(('$').__ne__, list(pd.DataFrame(net_inc.values[0].T).dropna().T.values[0])))
        
        Gross_profit = Income_Sheet[(Income_Sheet[Income_Sheet.columns[0]].str.contains('Gross profit')|
                                    Income_Sheet[Income_Sheet.columns[0]].str.contains('Gross margin')) ]
        
        if len(Gross_profit)!=0:
            Gross_profit  = list(filter(('$').__ne__, list(pd.DataFrame(Gross_profit.values[0].T).dropna().T.values[0])))
            Gross_profit=[i.replace('(','-').replace(')','').replace(',','') for i in Gross_profit]
            Gross_profit=[i for i in Gross_profit if i!='']
            Pscore+=1 if float(Gross_profit[1])-float(Gross_profit[2])>0 else 0

        Total_revenues = Income_Sheet[(Income_Sheet[Income_Sheet.columns[0]].str.contains('Total revenues') ) ]
        Total_revenues  = list(filter(('$').__ne__, list(pd.DataFrame(Total_revenues.values[0].T).dropna().T.values[0])))
        
        
        
        net_inc=[i.replace('(','-').replace(')','').replace(',','') for i in net_inc]
        net_inc=[net_inc[i] for i in range(len(net_inc)) if net_inc[i]!='' and net_inc[i] not in net_inc[i+1:]]
        
        Pscore+=2 if float(net_inc[1]) >0 else 0

        Pscore+=1 if float(oper_act[1])>0 else 0

        Pscore+=1 if float(oper_act[1])>float(net_inc[1]) else 0

    
    if len(Balance_Sheet)!=0 and len(Income_Sheet)!=0:
        if len(assets)>3:
            if float(Total_revenues[1])/(float(assets[1])+float(assets[2]))> float(Total_revenues[2])/(float(assets[3])+float(assets[2])):
                Pscore+=1
        else:
            if float(Total_revenues[1])/(float(assets[1])+float(assets[1]))> float(Total_revenues[2])/(float(assets[2])+float(assets[2])):
                Pscore+=1
    df['F_Score'].values[i]=Pscore
    

https://www.sec.gov/Archives/edgar/data/1001385/000119312508058830/d10k.htm 1
https://www.sec.gov/Archives/edgar/data/1001606/000110465908017372/a08-2371_110k.htm 1
https://www.sec.gov/Archives/edgar/data/1001614/000100161408000019/tenk_march28.htm 0
https://www.sec.gov/Archives/edgar/data/1001902/000095013408004891/f38725e10vk.htm 1
https://www.sec.gov/Archives/edgar/data/1001916/000119312508047834/d10k.htm 4
https://www.sec.gov/Archives/edgar/data/1001917/000095013408005261/v37363e10vk.htm 0
https://www.sec.gov/Archives/edgar/data/1002520/000116923208001513/d73968_10k.htm 0
https://www.sec.gov/Archives/edgar/data/1002554/000100255408000019/form10-k.htm 0
https://www.sec.gov/Archives/edgar/data/1002637/000104746908002055/a2183043z10-k.htm 0
https://www.sec.gov/Archives/edgar/data/1002663/000089161808000049/f37115e10vk.htm 1
https://www.sec.gov/Archives/edgar/data/1002910/000095013708003079/c23635e10vk.htm 0
https://www.sec.gov/Archives/edgar/data/1003214/000089161808000126/f38342e10vk

IndexError: list index out of range

In [28]:
# Total_revenues
# net_inc
# Gross_profit
# oper_act
# assets
# equity
# retained
# current_assets
# current_lia
# long_debt

# Balance_Sheet
# Income_Sheet
# Cash_flow

In [ ]:
# long_debt